In [1]:
import sqlite3, config
import pandas as pd

In [2]:
conn = sqlite3.connect(config.DB_FILE)
cursor = conn.cursor()

In [ ]:
q = """
    SELECT *
    FROM stock
    LIMIT 20
    """

df = pd.read_sql(q, conn)
df.head()

## Adding backtest_data column to stock table

Adding a backtest_data column to the stock table. It will be a simple boolean column.

This will be used to populate a list of available stocks that can be used when testing strategies on the backtesting page.

In [ ]:
# cursor.execute("""
#     ALTER TABLE stock
#     ADD backtest_data BOOLEAN NOT NULL
#     DEFAULT 0;
# """)

confirm 

In [ ]:
df = pd.read_sql(q, conn)
df.head()

### Update backtest_data 

Eventually will add this update to the populate_minute_date script

In [ ]:
q = """
    SELECT DISTINCT stock_id
    FROM stock_price_minute
    """

df = pd.read_sql(q, conn)
df.head()

In [ ]:
backtest_stocks = df.stock_id.drop_duplicates().tolist()
len(backtest_stocks)

In [ ]:
backtest_stocks = tuple(backtest_stocks)

In [ ]:
q = """
    SELECT id, name
    FROM stock
    WHERE id IN {}
    """.format(backtest_stocks)

df = pd.read_sql(q, conn)
df.head()

UPDATE backtest_data column:

In [ ]:
cursor.execute("""
    UPDATE stock
    SET backtest_data = 1
    WHERE id IN {}
""".format(backtest_stocks))

In [ ]:
conn.commit()

confirm

In [ ]:
q = """
    SELECT id, name, backtest_data
    FROM stock
    WHERE backtest_data = 1
    """

df = pd.read_sql(q, conn)
df.head(20)

## Create backtest_config table

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS backtest_config (
        run_id INTEGER PRIMARY KEY,
        date NOT NULL,
        stock_id INTEGER,
        strategy NOT NULL,
        bt_start NOT NULL,
        bt_end NOT NULL,
        set_cash INTEGER NOT NULL,
        open_range INTEGER NOT NULL,
        liquidate_time INTEGER NOT NULL,
        FOREIGN KEY (stock_id) REFERENCES stock (id)
    )
""")

In [ ]:
conn.commit()

In [ ]:
q = """
    SELECT *
    FROM backtest_config
    """

df = pd.read_sql(q, conn)
df.head(20)

## Create backtest_report table

In [3]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS backtest_report (
        run_id INTEGER,
        start_cash INTEGER,
        rpl REAL,
        result_won_trades REAL,
        result_lost_trades REAL,
        profit_factor REAL,
        rpl_per_trade REAL,
        total_return REAL,
        annual_return REAL,
        max_money_drawdown REAL,
        max_pct_drawdown REAL,
        total_number_trades INTEGER,
        trades_closed INTEGER,
        pct_winning REAL,
        pct_losing REAL,
        avg_money_winning REAL,
        avg_money_losing REAL,
        best_winning_trade REAL,
        worst_losing_trade REAL,
        sharpe_ratio REAL,
        sqn_score REAL,
        sqn_human TEXT,
        FOREIGN KEY(run_id) REFERENCES backtest_config(run_id)
    )
""")

In [4]:
conn.commit()

In [5]:
q = """
    SELECT *
    FROM backtest_report
    """

df = pd.read_sql(q, conn)
df.head(20)

,run_id,start_cash,rpl,result_won_trades,result_lost_trades,profit_factor,rpl_per_trade,total_return,annual_return,max_money_drawdown,...,trades_closed,pct_winning,pct_losing,avg_money_winning,avg_money_losing,best_winning_trade,worst_losing_trade,sharpe_ratio,sqn_score,sqn_human
